# Classifying high dimensionality remote sensing images

## Intro to hyperspectral remote sensing
Insert graphic of cube, explanative text

In [1]:
from arcgis.gis import GIS
from arcgis import raster

In [2]:
# gis = GIS("https://dev0001902.esri.com/portal","admin", verify_cert=False)
gis = GIS("https://pythonapi.playground.esri.com/portal","atma.mani")
# gis = GIS("https://python.playground.esri.com/portal","arcgis_python","amazing_arcgis_123")

Enter password: ········


In [3]:
# aviris_item = gis.content.get('596098ec0e834e7d9f8e81e14de14ef2')
aviris_item = gis.content.search('AVIRIS', item_type='Imagery Layer')[0]
aviris_item

<Item title:"omaha_aviris" type:Imagery Layer owner:atma.mani>

In [4]:
map1 = gis.map('Omaha, NE')
map1

In [6]:
map1.add_layer(aviris_item)

## Draw using false color composite

### Get AVIRIS band wavelengths

In [7]:
aviris_layer = aviris_item.layers[0]
aviris_bands = aviris_layer.key_properties()['BandProperties']
len(aviris_bands)

224

In [10]:
aviris_bands[0]['wavelength']

'365.9298'

In [11]:
aviris_bands[-1]['wavelength']

'2497.036'

In [12]:
aviris_layer = aviris_item.layers[0]
aviris_fcc = raster.stretch(raster = raster.extract_band(aviris_layer, band_wavelengths=[831, 665.0994, 560]),
                            stretch_type='histogram')
map1.remove_layers()
map1.add_layer(aviris_fcc)

## Get spectral signature from the remotely sensed image

In [13]:
map2 = gis.map('Greenwood, NE', zoomlevel=13)
map2

0


KeyError: 'values'

0


KeyError: 'values'

0
[{'location': {'x': -10740063.886278052, 'y': 5005786.460497638, 'spatialReference': {'wkid': 102100, 'latestWkid': 3857}}, 'locationId': 0, 'value': '1078 1207 1260 1526 2169 2303 2185 2101 2322 2460 2424 2361 2302 2200 2170 2123 2077 2125 2173 2178 2135 2076 2062 1993 2002 2007 1971 1923 1964 1933 1912 1982 1902 1971 1984 1818 1875 2057 2056 1928 2232 2507 2473 1609 2392 2552 2463 2421 2241 1972 2098 2297 2320 2319 2302 2298 2215 1827 1642 1562 1391 755 718 937 1443 1619 1906 1982 1942 1957 1933 1909 1880 1836 1771 1735 1643 1548 1300 893 322 302 450 428 761 1087 1164 1166 1216 1198 1263 1322 1376 1373 1315 1128 1288 681 535 523 505 390 300 341 143 88 10 2 2 2 4 3 31 48 58 68 146 169 135 194 331 420 1756 1492 1440 1278 1370 1372 1187 1133 1078 931 900 907 881 822 835 847 847 850 881 879 877 831 799 693 718 679 627 506 390 174 52 16 1 0 1 2 2 2 1 1 3 2 3 3 6 18 54 86 302 590 755 461 166 290 739 966 791 591 644 839 954 989 1009 1000 1022 1041 999 951 918 918 875 844 882 910 897 852 8

KeyError: 'values'

In [14]:
map2.add_layer(aviris_fcc)

##### drawing logic

In [15]:
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
from IPython.display import clear_output
output_notebook()

Loading BokehJS ...

In [16]:
symbol_list = [{"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueA.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueB.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueC.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueD.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueE.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueF.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueG.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueH.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueI.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueJ.png","contentType":"image/png","width":14,"height":22},
              {"angle":0,"xoffset":0,"yoffset":8.15625,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/AtoZ/blueK.png","contentType":"image/png","width":14,"height":22}]
legend_list = ['A','B','C','D','E','F','G','H','I','J']
bokeh_colors=['blue','blueviolet','brown',
              'burlywood','cadetblue','chartreuse',
              'chocolate','coral','cornflowerblue',
             'cyan','darkblue']
x_axis = [float(s['wavelength']) for s in aviris_bands]

In [18]:
i = 0
p = figure(title="Spectral Profile", x_axis_label='Wavelengths', 
           y_axis_label='Radiance', width=600, height=300)

def get_samples(map_handle, clicked_point):
    global i, p
    print(i)
    if i>0:
        clear_output()

    # Get cell value from imagery layer
    image_data = aviris_layer.get_samples(clicked_point)
    print(image_data)
    radiance_values = image_data[0]['values']

    # choose symbol
    map_handle.draw(clicked_point, symbol = symbol_list[i])
    p.line(x_axis, radiance_values, legend=legend_list[i], line_color=bokeh_colors[i], line_width=2)
    show(p)
    
    i = i+1
    print(i)
    
map2.on_click(get_samples)

In [69]:
legend_list[i]

'D'